# Verify Credential

This notebook illustrates how a VC signed using a BIP 322 signature could be parsed and verified by a verifier.

## Notebook Author

* Will Abramson
* [Legendary Requirements](http://legreq.com/)
* Contact: will@legreq.com

## Acknowledgements

This work was funded by Ryan Grant and [Digital Contract Design](https://contract.design/). Thanks also go to Joe Andrieu, Kalle Alm, Pieter Wuille and Jimmy Song for engaging with and supporting various aspects of this work.

## Steps

1. Parse signature from signed VC
2. Parse/resolve bitcoin address used the sign vc
3. Remove proof from signed VC 
4. Canonicalize VC
5. Verify signatue on VC

In [1]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


## 1. Parse signature from signed VC

**YOU SHOULD COPY THE SIGNED VC PRODUCED IN the [issue_credential](http://localhost:8888/notebooks/vc/issue_credential.ipynb) NOTEBOOK**

In [3]:
import json
signed_vc = {"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "did:btcr:xyv2-xzpq-q9wa-p7t", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d'Universit\u00e9", "lang": "fr"}]}}, "proof": {"type": "BIP322Signature2022", "created": "2022-05-23T21:19:10Z", "proofPurpose": "assertionMethod", "verificationMethod": "did:btcr:xyv2-xzpq-q9wa-p7t#vckey-1", "bip322_sig": "AkgwRQIhAOnhdQ6NK73jWCSiEXgXOXJxoG2LeZwQ9KZVAbPr0/ALAiAs/Jzx5bp4voCDSUpPPrlpZM8gw/yO7jPq67VURxEXiAEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x"}}
signed_vc

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'id': 'http://example.edu/credentials/1872',
 'type': ['VerifiableCredential', 'AlumniCredential'],
 'issuer': 'did:btcr:xyv2-xzpq-q9wa-p7t',
 'issuanceDate': '2010-01-01T19:23:24Z',
 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21',
  'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1',
   'name': [{'value': 'Example University', 'lang': 'en'},
    {'value': "Exemple d'Université", 'lang': 'fr'}]}},
 'proof': {'type': 'BIP322Signature2022',
  'created': '2022-05-23T21:19:10Z',
  'proofPurpose': 'assertionMethod',
  'verificationMethod': 'did:btcr:xyv2-xzpq-q9wa-p7t#vckey-1',
  'bip322_sig': 'AkgwRQIhAOnhdQ6NK73jWCSiEXgXOXJxoG2LeZwQ9KZVAbPr0/ALAiAs/Jzx5bp4voCDSUpPPrlpZM8gw/yO7jPq67VURxEXiAEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x'}}

In [4]:
proof = signed_vc["proof"]
bip322_sig = proof["bip322_sig"]

In [5]:
print("bip322 signature in VC : \n", bip322_sig)

bip322 signature in VC : 
 AkgwRQIhAOnhdQ6NK73jWCSiEXgXOXJxoG2LeZwQ9KZVAbPr0/ALAiAs/Jzx5bp4voCDSUpPPrlpZM8gw/yO7jPq67VURxEXiAEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x


## 2. Resove Issuer did:btcr and parse for VC verificationMethod used to sign VC

This verificationMethod should be a bitcoin address such as a p2wpkh.

### 2.1 Resolve issuer did:btcr to associated DID document

This needs to be defined.

In [6]:
issuer_did_btcr = signed_vc["issuer"]

In [7]:
# TODO: Actually define/implement resolution process
resolved_did_doc = {
    "@context": ["https://w3id.org/did/v0.11", "https://w3id.org/btcr/v1"],
    "id": "did:btcr:xyv2-xzpq-q9wa-p7t",
    "verificationMethod": [
       {
            "id": "did:btcr:xyv2-xzpq-q9wa-p7t#satoshi",
            "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
            "type": "EcdsaSecp256k1VerificationKey2019",
            "publicKeyBase58": "owh12LKNuphe97teJTZKQTKNewSVTwjHcskPbq34epCY"
        },
        {
             "id": "did:btcr:xyv2-xzpq-q9wa-p7t#vckey-0",
             "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
             "type": "EcdsaSecp256k1VerificationKey2019",
             "publicKeyBase58": "owh12LKNuphe97teJTZKQTKNewSVTwjHcskPbq34epCY"
         },
    ##########################      EXAMPLE       #################################
        ### What are the resolution rules that parse the Bitcoin Tx at ref xyv2-xzpq-q9wa-p7t
        ### To populate this verificationMethod into the didDocument.
        ### E.g. scriptPubKey from UTX0 at index 1 in tx at ref xyv2-xzpq-q9wa-p7t
        {
             "id": "did:btcr:xyv2-xzpq-q9wa-p7t#vckey-1",
             "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
             "type": "BIP0322",
             "address": "bc1qz52z3pe9fg3qxv9n6yhxgj7rcn8wsvpq56v9ck"
        }
    
    ###############################################################################
    ],
    "authentication": ["#satoshi"],
    "assertionMethod": ["#vckey-0","#vckey-1"]
}


### 2.2. Parse Resolved DID document for verificationMethod defined in proof

In [8]:
verificationMethod = signed_vc["proof"]["verificationMethod"]
print(verificationMethod)

did:btcr:xyv2-xzpq-q9wa-p7t#vckey-1


In [9]:
address = None
for vm in resolved_did_doc["verificationMethod"]:
    if vm["id"] == verificationMethod:
        address = vm["address"]
        break;
        
print("Bitcoin address to use in BIP 322 Verification : ", address)

Bitcoin address to use in BIP 322 Verification :  bc1qz52z3pe9fg3qxv9n6yhxgj7rcn8wsvpq56v9ck


## 3. Remove proof from signed_vc payload

In [10]:
vc = signed_vc
del vc["proof"]

vc

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'id': 'http://example.edu/credentials/1872',
 'type': ['VerifiableCredential', 'AlumniCredential'],
 'issuer': 'did:btcr:xyv2-xzpq-q9wa-p7t',
 'issuanceDate': '2010-01-01T19:23:24Z',
 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21',
  'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1',
   'name': [{'value': 'Example University', 'lang': 'en'},
    {'value': "Exemple d'Université", 'lang': 'fr'}]}}}

## 4. Canonicalize VC payload

**Should we use URDNA canonicalization?**

In [12]:
from pyld import jsonld

In [15]:
normalized = jsonld.normalize(
    vc, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})

normalized

'<did:example:c276e12ec21ebfeb1f712ebc6f1> <http://schema.org/name> _:c14n0 .\n<did:example:c276e12ec21ebfeb1f712ebc6f1> <http://schema.org/name> _:c14n1 .\n<did:example:ebfeb1f712ebc6f1c276e12ec21> <http://schema.org/alumniOf> <did:example:c276e12ec21ebfeb1f712ebc6f1> .\n<http://example.edu/credentials/1872> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://example.org/examples#AlumniCredential> .\n<http://example.edu/credentials/1872> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://www.w3.org/2018/credentials#VerifiableCredential> .\n<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#credentialSubject> <did:example:ebfeb1f712ebc6f1c276e12ec21> .\n<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#issuanceDate> "2010-01-01T19:23:24Z"^^<http://www.w3.org/2001/XMLSchema#dateTime> .\n<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#issuer> <did:btcr:xyv2-xzpq-q9wa-p7t> .\n'

## 5. Verify bip322 signature on VC

In [16]:
from src.message import verify_message

### NOTE: the output `"mismatch between length and consumed bytes 88 vs 31"` is expected" and occurs when attempting to parse the signature as a full Bitcoin transaction. See is_full_signature in [src/message.py](../../edit/src/message.py) and the [script.py](https://github.com/buidl-bitcoin/buidl-python/blob/e62e0b9fd4271473cf641cde0fe366e65c787403/buidl/script.py#L109) file from Buidl Python

In [18]:

verification_result = verify_message(address, bip322_sig, normalized)

print("Signature is verified : ", verification_result)

mismatch between length and consumed bytes 83 vs 44
Signature is verified :  True


## Verify Legacy Signature on VC

Note: this is incomplete. Somehow need to be able to retrieve the Secp256k1 ECC point associated with the p2pkh_address used to sign the message. Currently unsure exactly how. Bitcoin core has a method privateKey.SignCompact which does this - possibly missing from buidl-python

Note: Legacy signatures are not recommened, rather it is recommended to encode p2pkh bip322 signatures using the full encoding.

In [13]:
legacy_signed_vc = {"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "https://example.edu/issuers/565049", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d'Universit\u00e9", "lang": "fr"}]}}, "proof": {"type": "BIP322Signature2022", "created": "2022-05-23T21:19:10Z", "proofPurpose": "assertionMethod", "verificationMethod": "https://example.edu/issuers/565049#key-1", "bip322_sig": "MEUCIQDLQvF+0Dg0uruDJCydBwCV8vC03Y4VagBnEzKflu1HNQIgSKBu1Q6xWlQSZZVxUPgzVnTrXakcdBvV4BdWS4Oi/Qk="}}
legacy_signed_vc

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'id': 'http://example.edu/credentials/1872',
 'type': ['VerifiableCredential', 'AlumniCredential'],
 'issuer': 'https://example.edu/issuers/565049',
 'issuanceDate': '2010-01-01T19:23:24Z',
 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21',
  'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1',
   'name': [{'value': 'Example University', 'lang': 'en'},
    {'value': "Exemple d'Université", 'lang': 'fr'}]}},
 'proof': {'type': 'BIP322Signature2022',
  'created': '2022-05-23T21:19:10Z',
  'proofPurpose': 'assertionMethod',
  'verificationMethod': 'https://example.edu/issuers/565049#key-1',
  'bip322_sig': 'MEUCIQDLQvF+0Dg0uruDJCydBwCV8vC03Y4VagBnEzKflu1HNQIgSKBu1Q6xWlQSZZVxUPgzVnTrXakcdBvV4BdWS4Oi/Qk='}}

In [14]:
proof = legacy_signed_vc["proof"]
bip322_sig = proof["bip322_sig"]
print("BIP 322 Legacy Signature", bip322_sig)

BIP 322 Legacy Signature MEUCIQDLQvF+0Dg0uruDJCydBwCV8vC03Y4VagBnEzKflu1HNQIgSKBu1Q6xWlQSZZVxUPgzVnTrXakcdBvV4BdWS4Oi/Qk=


In [15]:
vc = legacy_signed_vc
del vc["proof"]

vc

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'id': 'http://example.edu/credentials/1872',
 'type': ['VerifiableCredential', 'AlumniCredential'],
 'issuer': 'https://example.edu/issuers/565049',
 'issuanceDate': '2010-01-01T19:23:24Z',
 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21',
  'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1',
   'name': [{'value': 'Example University', 'lang': 'en'},
    {'value': "Exemple d'Université", 'lang': 'fr'}]}}}

In [16]:
vc_payload = json.dumps(vc)

In [17]:
verification_result = verify_message(address, bip322_sig, vc_payload)

Is taproot False
Signature is not full
Signature is niether an encoded witness or full transaction. Fall back to legacy


NotImplementedError: TODO

In [ ]:
from buidl.helper import base64_decode

der_sig = base64_decode(bip322_sig)

len(der_sig)

# str_to_bytes(bip322_sig)

In [ ]:
from buidl.ecc import Signature

In [ ]:
Signature.parse(base64_decode(bip322_sig)).der().hex()

## Verify BIP 322 Full Signature on the VC

Signature created using a p2pkh address

In [18]:
p2pkh_address = "12vTLer5v6RyMpnxetu7mGxtveRsVSps5R"

In [19]:
full_signed_vc = {"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "https://example.edu/issuers/565049", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d'Universit\u00e9", "lang": "fr"}]}}, "proof": {"type": "BIP322Signature2022", "created": "2022-05-23T21:19:10Z", "proofPurpose": "assertionMethod", "verificationMethod": "https://example.edu/issuers/565049#key-1", "bip322_sig": "AAAAAAABAar/MRPVK4zsVrD/SuHuzaw0Ki4QBvsUHTk6+RSUe4f/AAAAAGpHMEQCIGQwJ85k5iVzzuUh7kyLejW4JUW+ZLTl2vcuA4TbIi07AiB0s13e3VuS/prBmbnu6togBAy4pPxbCvU1iMp6O32newEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8xAAAAAAEAAAAAAAAAAAFqAAAAAAA="}}

full_signed_vc

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'id': 'http://example.edu/credentials/1872',
 'type': ['VerifiableCredential', 'AlumniCredential'],
 'issuer': 'https://example.edu/issuers/565049',
 'issuanceDate': '2010-01-01T19:23:24Z',
 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21',
  'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1',
   'name': [{'value': 'Example University', 'lang': 'en'},
    {'value': "Exemple d'Université", 'lang': 'fr'}]}},
 'proof': {'type': 'BIP322Signature2022',
  'created': '2022-05-23T21:19:10Z',
  'proofPurpose': 'assertionMethod',
  'verificationMethod': 'https://example.edu/issuers/565049#key-1',
  'bip322_sig': 'AAAAAAABAar/MRPVK4zsVrD/SuHuzaw0Ki4QBvsUHTk6+RSUe4f/AAAAAGpHMEQCIGQwJ85k5iVzzuUh7kyLejW4JUW+ZLTl2vcuA4TbIi07AiB0s13e3VuS/prBmbnu6togBAy4pPxbCvU1iMp6O32newEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8xAAAAAAEAAAAAAAAAAAFqAAAAAAA='}}

In [20]:
proof = full_signed_vc["proof"]
bip322_sig = proof["bip322_sig"]
print("BIP 322 FULL Signature", bip322_sig)

BIP 322 FULL Signature AAAAAAABAar/MRPVK4zsVrD/SuHuzaw0Ki4QBvsUHTk6+RSUe4f/AAAAAGpHMEQCIGQwJ85k5iVzzuUh7kyLejW4JUW+ZLTl2vcuA4TbIi07AiB0s13e3VuS/prBmbnu6togBAy4pPxbCvU1iMp6O32newEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8xAAAAAAEAAAAAAAAAAAFqAAAAAAA=


In [21]:
vc = full_signed_vc
del vc["proof"]

vc

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'id': 'http://example.edu/credentials/1872',
 'type': ['VerifiableCredential', 'AlumniCredential'],
 'issuer': 'https://example.edu/issuers/565049',
 'issuanceDate': '2010-01-01T19:23:24Z',
 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21',
  'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1',
   'name': [{'value': 'Example University', 'lang': 'en'},
    {'value': "Exemple d'Université", 'lang': 'fr'}]}}}

In [22]:
import json
vc_payload = json.dumps(vc)

In [23]:
verification_result = verify_message(p2pkh_address, bip322_sig, vc_payload)

Is taproot False


In [24]:
verification_result

True